# Lab Four: The Multi-Layer Perceptron

#### *Harrison Noble, Henry Lambson & Jack Schipper*

## 1 Load, Split & Balance

### 1.1 Load Data & Save to Pandas Data Frame

- __[.5 points]__ Load the data into memory and save it to a pandas data frame. Do not normalize or one-hot encode any of the variables until asked to do so.  Remove any observations that having missing data. Encode any string data as integers for now. 

In [1]:
#TODO

### 1.2 Split Dataset

- __[.5 points]__ Assume you are equally interested in the classification performance for each class in the dataset. Split the dataset into 80% for training and 20% for testing.

In [2]:
#TODO

### 1.3 Balance Dataset

- __[.5 points]__ Balance the dataset so that about the same number of instances are within each class. Choose a method for balancing the dataset and explain your reasoning for selecting this method. One option is to choose quantization thresholds for the "ChildPoverty" variable that equally divide the data into four classes. *Should balancing of the dataset be done for both the training and testing set? Explain.*

In [3]:
#TODO

## 2 Pre-Processing

### 2.1 Use Example Two-Layer Perceptron Network & Quantify Performance

- __[.5 points]__ Use the example two-layer perceptron network from the class example and quantify performance using accuracy. Do not normalize or one-hot encode the data (not yet). Be sure that training converges by graphing the loss function versus the number of epochs. 

In [4]:
#TODO

### 2.2 Normalize Continuous Numeric Feature Data

- __[.5 points]__ Now normalize the continuous numeric feature data. Use the example two-layer perceptron network from the class example and quantify performance using accuracy. Be sure that training converges by graphing the loss function versus the number of epochs. 

In [5]:
#TODO

### 2.3 One Hot Encode Categorical Data & Normalize Continuous Numeric Features

- __[.5 points]__ Now normalize the continuous numeric feature data AND one hot encode the categorical data. Use the example two-layer perceptron network from the class example and quantify performance using accuracy. Be sure that training converges by graphing the loss function versus the number of epochs. 

In [6]:
#TODO

### 2.4 Compare Performance of the Three Trained Models

- __[1 points]__ Compare the performance of the three models you just trained. Are there any meaningful differences in performance? Explain, in your own words, why these models have (or do not have) different performances.  
    - *Use one-hot encoding and normalization on the dataset for the remainder of this lab assignment.*

In [7]:
#TODO

## 3 Modeling

### 3.1 Add Support for Third Layer in Multi-Layer Perceptron

- __[1 points]__ Add support for a third layer in the multi-layer perceptron. Add support for saving (and plotting after training is completed) the average magnitude of the gradient for each layer, for each epoch. For magnitude calculation, you are free to use either the average absolute values or the L1/L2 norm. Quantify the performance of the model and graph the magnitudes for each layer versus the number of epochs.

In [8]:
#TODO

### 3.2 Add Support for Fourth Layer

- __[1 points]__ Repeat the previous step, adding support for a fourth layer.

In [9]:
#TODO

### 3.3 Add Support for Fifth Layer

- __[1 points]__ Repeat the previous step, adding support for a fifth layer. 

In [10]:
#TODO

### 3.4 Implement Adaptive Learning Technique on Five Layer Network

- __[2 points]__ Implement an adaptive learning technique that was discussed in lecture and use it on the five layer network. Compare the performance of this model with and without the adaptive learning strategy. Do not use AdaM for the adaptive learning technique. 

In [11]:
#TODO

## Exceptional Work: _______

- 5000 level student: You have free reign to provide additional analyses.
- One idea __(required for 7000 level students)__:  Implement adaptive momentum (AdaM) in the five layer neural network and quantify the performance. 

In [ ]:
#TODO